In [ ]:
## CPE 4903 - Prediction of Time Series: RNN ##
Predict [y(t+n)] given x(t-time_steps)...x(t), x(t) = [1 2 3 4 5 4 3 2 1 2 3 ...] sawtooth

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, LSTM, SimpleRNN
from keras.models import Model, Sequential
import pandas as pd
#import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from keras.preprocessing.sequence import TimeseriesGenerator

def plot_loss(history, title):
# summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss: ' + title)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.show()
    print('MSE val = ', history.history['val_loss'][-1])

def plot_y_yhat(x_test, y_test, model, title):
#y_pred = model.predict(X_test)
    y_hat = model.predict(x_test)
    Time_test = np.arange(0, len(y_hat))
    plt.figure(figsize=(20,10))
    plt.plot(Time_test, y_hat, '-+', Time_test, y_test)
    plt.legend(['Predicted', 'True'], loc='upper right')
    plt.title('Y vs Yhat: ' + title)
    results = model.evaluate(x_test, y_test)
    print("test loss, test acc:", results)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

if True: #sawtooth waveform
    x = np.array([1, 2, 3, 4, 5, 4, 3, 2, 1, 2, 3, 4, 5, 4, 3, 2, 1, 2, 3, 4, 5, 4, 3, 2, 1, 2, 3, 4, 5, 4])
    x = x.reshape(-1,1)
else:
    time = np.arange(0, 100, 0.1)
    signal = np.sin(time) * np.sin(5*time + .25) + np.sin(.5*time + .35)
    noise = 0*np.random.normal(scale=0.5, size=len(time))
    data = signal + noise
    x = data
    x = x.reshape(-1,1)


x.shape

x.T

plt.plot(x)

train_size = int(len(x) * 0.7)
test_size = len(x) - train_size
train = x[:train_size]
test = x[train_size:]

train = train.reshape(-1,1)
test = test.reshape(-1,1)
print(train.shape, test.shape)

# simple scaling
#train = train/np.max(train)
#test = test/np.max(test)

print(train)

![image.png](attachment:image.png)

![image-2.png](attachment:image-2.png)

#train = data
time_steps =4
n = 1 # prediction y(t+n) ahead


# Train sets
m_train = len(train)

# TimeseriesGenerator generates matrix X_train with shape (m_train-time_steps-n+1, time_steps)
# y_train with shape ((m_train-time_steps-n+1, 1)

target = np.roll(train,1-n) # pre shift left n-1 
dataset = TimeseriesGenerator(train, target , length=time_steps, batch_size = m_train-time_steps-n+1)
X_train, y_train = dataset[0]  # first sample X_train[0] = train[0..time_steps-1], y_train = [time_steps+n-1] n step ahead 

y_train = y_train.reshape(-1,1)
print('X_train and y_train shapes: ', [X_train.shape, y_train.shape])

# Test sets
m_test = len(test)
target = np.roll(test,1-n)
dataset = TimeseriesGenerator(test, target , length=time_steps, batch_size = m_test-time_steps-n+1)
X_test, y_test = dataset[0]
y_test = y_test.reshape(-1,1)
print('X_test and y_test shapes: ', [X_test.shape, y_test.shape])


X_train

y_train

print(X_train)

y_train.shape

print(y_train)

x

m = len(y_train)
nx = 1 # dim of x<t>
print('m, time_steps, n = look_ahead, nx: ', m, time_steps, n, nx)



# RNN #
hidden_neurons = 64
model = Sequential()
model.add(SimpleRNN(units=hidden_neurons, input_shape=(time_steps, nx)))
model.add(Dense(units=200,activation='relu'))
model.add(Dense(units=1, activation=None))
opt=keras.optimizers.Adam()

model.compile(loss='mean_squared_error',optimizer=opt, metrics = ['accuracy'])
model.summary()

history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)

yhat_test = model.predict(X_test)

plot_loss(history, 'RNN')

plot_y_yhat(X_test, y_test, model, 'RNN')   # runs model.predict in function

#train = data
time_steps =4
n = 2 # prediction y(t+n) ahead


# Train sets
m_train = len(train)

# TimeseriesGenerator generates matrix X_train with shape (m_train-time_steps-n+1, time_steps)
# y_train with shape ((m_train-time_steps-n+1, 1)

target = np.roll(train,1-n) # pre shift left n-1 
dataset = TimeseriesGenerator(train, target , length=time_steps, batch_size = m_train-time_steps-n+1)
X_train, y_train = dataset[0]  # first sample X_train[0] = train[0..time_steps-1], y_train = [time_steps+n-1] n step ahead 

y_train = y_train.reshape(-1,1)
print('X_train and y_train shapes: ', [X_train.shape, y_train.shape])

# Test sets
m_test = len(test)
target = np.roll(test,1-n)
dataset = TimeseriesGenerator(test, target , length=time_steps, batch_size = m_test-time_steps-n+1)
X_test, y_test = dataset[0]
y_test = y_test.reshape(-1,1)
print('X_test and y_test shapes: ', [X_test.shape, y_test.shape])

m = len(y_train)
nx = 1 # dim of x<t>
print('m, time_steps, n = look_ahead, nx: ', m, time_steps, n, nx)



# RNN #
hidden_neurons = 64
model = Sequential()
model.add(SimpleRNN(units=hidden_neurons, input_shape=(time_steps, nx)))
model.add(Dense(units=200,activation='relu'))
model.add(Dense(units=1, activation=None))
opt=keras.optimizers.Adam()

model.compile(loss='mean_squared_error',optimizer=opt, metrics = ['accuracy'])
model.summary()

history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)

plot_loss(history, 'RNN')

plot_y_yhat(X_test, y_test, model, 'RNN')   # runs model.predict in function

# As we may see from the two graphs when n is increased to n = 2 our predictions became more accurate. Although it appears
# that before epoch 50 n=1 is more stable and accurate for the train vs the validations. This can be seen in the loss chart.
# However, after 200 iterations the loss for n=2 was significantly less than that of n=1. 